In [1]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from ff_energy.ffe.utils import read_from_pickle
import jax.numpy as jnp
from ff_energy.ffe.plot import plot_energy_MSE, plot_ff_fit
from ff_energy.ffe.structure import atom_key_pairs
from ff_energy.ffe.potential import LJ, akp_indx
from ff_energy.ffe.ff import FF
from ff_energy.ffe.ff_fit import LJ_bound, load_ff, fit_func, fit_repeat
from ff_energy.ffe.data import pairs_data
from ff_energy.ffe.utils import pickle_output, read_from_pickle, str2int




In [15]:
sig_bound = (0.001, 2.5)
ep_bound = (0.001, 2.5)
chg_bound = (100,2000)

CHGPEN_bound = [(chg_bound),(chg_bound),(chg_bound),(chg_bound),(0,2000)]
LJ_bound = ((sig_bound), (sig_bound), (ep_bound), (ep_bound))
DE_bound = ((sig_bound), (sig_bound), (ep_bound), (ep_bound), 
            (1, 8), (6, 20))

In [3]:
#  example
# ffpkl = "ECOL_pbe0dz_FOE.ff.pkl"

In [4]:
# Parameters
ffpkl = "ECOL_pbe0dz_clusters.ff.pkl"


#  LJ fit


In [16]:
_c = next(read_from_pickle(f"ff/{ffpkl}"))
_c

FF: LJ water_cluster ECOL Exact intE (jax_coloumb: False)

In [6]:
# _c.targets

In [26]:
fit_repeat(_c, 
           3, 
           f"{ffpkl}_LJ",
           bounds = LJ_bound,
           loss="jax",
           quiet='true'
          )

/home/boittier/Documents/phd/ff_energy/ff_energy/ffe/ff_fit.py:175: OptimizeWarning: Unknown solver options: pgtol
  res = minimize(


In [27]:
pd.DataFrame(_c.opt_results).sort_values("fun")

,fun,nit,nfev,status,success,message,x,final_simplex
5,319.147644,432,743,0,True,Optimization terminated successfully.,"[0.001, 2.5, 0.001, 0.0075410345500456465]","([[0.001, 2.5, 0.001, 0.0075410345500456465], ..."
3,319.147736,707,1217,0,True,Optimization terminated successfully.,"[0.001, 2.499998213954245, 0.001, 0.0075409834...","([[0.001, 2.499998213954245, 0.001, 0.00754098..."
2,319.149139,487,827,0,True,Optimization terminated successfully.,"[0.001, 2.5, 0.0010018543709556312, 0.00754160...","([[0.001, 2.5, 0.0010018543709556312, 0.007541..."
1,458.764771,306,563,0,True,Optimization terminated successfully.,"[0.002564546225299443, 2.2290969870233797, 2.5...","([[0.002564546225299443, 2.2290969870233797, 2..."
4,1044.884277,336,607,0,True,Optimization terminated successfully.,"[1.864439123517367, 0.9251054027770511, 0.0014...","([[1.864439123517367, 0.9251054027770511, 0.00..."
0,19700.291016,52,160,0,True,Optimization terminated successfully.,"[0.001, 0.001, 2.5, 0.3448839818311519]","([[0.001, 0.001, 2.5, 0.3448839818311519], [0...."


In [25]:
list(pd.DataFrame(_c.opt_results).sort_values("fun").x)[0]

array([1.00000000e-03, 2.50000000e+00, 1.00185437e-03, 7.54160033e-03])

#  DE fit


In [19]:
DEFF = next(read_from_pickle(f"ff/{ffpkl}"))

In [20]:
fit_repeat(DEFF, 
           10, 
           "test",
           bounds = DE_bound,
           loss="jax_de",
           quiet='true'
          )

/home/boittier/Documents/phd/ff_energy/ff_energy/ffe/ff_fit.py:175: OptimizeWarning: Unknown solver options: pgtol
  res = minimize(


In [21]:
pd.DataFrame(DEFF.opt_results).sort_values("fun")

,fun,nit,nfev,status,success,message,x,final_simplex
4,226.926987,1752,2762,0,True,Optimization terminated successfully.,"[2.1757148888287796, 2.473551545521282, 0.0010...","([[2.1757148888287796, 2.473551545521282, 0.00..."
5,251.949478,975,1600,0,True,Optimization terminated successfully.,"[2.0845429263144917, 2.495445564153819, 0.0096...","([[2.0845429263144917, 2.495445564153819, 0.00..."
9,279.912750,624,1040,0,True,Optimization terminated successfully.,"[2.4942896889519117, 0.5719099627239144, 0.005...","([[2.4942896889519117, 0.5719099627239144, 0.0..."
3,281.509338,772,1235,0,True,Optimization terminated successfully.,"[0.001, 2.4999998519867717, 0.0092497631059851...","([[0.001, 2.4999998519867717, 0.00924976310598..."
6,292.009277,650,1098,0,True,Optimization terminated successfully.,"[2.5, 0.8630909844902293, 0.00926175520186966,...","([[2.5, 0.8630909844902293, 0.0092617552018696..."
1,297.859741,570,946,0,True,Optimization terminated successfully.,"[0.0010033470376129478, 2.4972993544838893, 2....","([[0.0010033470376129478, 2.4972993544838893, ..."
0,19700.291016,49,216,0,True,Optimization terminated successfully.,"[0.06082497801368181, 0.22659003180896642, 0.4...","([[0.06082497801368181, 0.22659003180896642, 0..."
2,19700.291016,52,216,0,True,Optimization terminated successfully.,"[0.001, 0.42714400746823733, 2.263224370064375...","([[0.001, 0.42714400746823733, 2.2632243700643..."
7,19700.291016,45,199,0,True,Optimization terminated successfully.,"[0.15167918190794294, 0.17523762649866076, 2.0...","([[0.15167918190794294, 0.17523762649866076, 2..."
8,19700.291016,74,254,0,True,Optimization terminated successfully.,"[0.07079822655673662, 0.2811234181313511, 2.5,...","([[0.07079822655673662, 0.2811234181313511, 2...."


In [24]:
list(pd.DataFrame(DEFF.opt_results).sort_values("fun").x)[0]

array([2.17571489e+00, 2.47355155e+00, 1.00921262e-03, 1.08052108e-02,
       7.99246300e+00, 1.09280242e+01])